In [1]:
import { SamplingMixins, causalNetSampling } from 'causal-net.sampling';
import { Platform } from 'causal-net.utils';
import * as Core from 'causal-net.core';
var { Tensor, Functor, causalNetCore } = Core;
console.log(causalNetSampling instanceof Functor);
var T = causalNetCore.CoreTensor;

false


http://mathfaculty.fullerton.edu/mathews/n2003/QRMethodMod.html

In [2]:
var m = T.tensor([ [ 1, 7,10, 5],
                   [ 7, 2, 3,50],
                   [10, 3, 3, 5], 
                   [ 5,50, 5, 5] ] );
function Eignh(m){
    const [s,_] = m.shape;
    diag = T.eye(s);
    var [q, r] = T.linalg.qr(m);
    var  cQ = T.eye(s);
    let err = q.pow(2).sum().sub(q.mul(diag).pow(2).sum());
    while( err.dataSync()[0] > 1e-4 ){
        [q, r] = T.linalg.qr(m);
        err = q.pow(2).sum().sub(q.mul(diag).pow(2).sum());
        m = r.dot(q);
        cQ = cQ.dot(q);
    }
    return [m.mul(diag).sum(1), cQ];
}

In [3]:
//calculate the determinant of a matrix m
function det(m) {
    return T.tidy(() => {
        let [D, _] = m.shape;
        
        if (D === 2) {
            const t = m.as1D()
            const a = t.slice([0], [1]).dataSync()[0];
            const b = t.slice([1], [1]).dataSync()[0];
            const c = t.slice([2], [1]).dataSync()[0];
            const d = t.slice([3], [1]).dataSync()[0];
            result = a * d - b * c;
            return result;

        } else {
            let s = 0, sub_m, sub_m_rc, d, a ;            
            const cols = [...Array(D).keys()];
            for(let c=0; c<D; c++){
                sub_m = m.gather(T.tensor1d(cols.filter(e => e !== c), 'int32'),1)
                sub_m_rc = sub_m.gather(T.tensor1d(cols.filter(e => e !== 0), 'int32'))    
                d = det(sub_m_rc);
                a = m.gather(0).gather(c).dataSync()[0];
                s += a*Math.pow(-1, c)* d;
            }
            
            return s;
        }
    })
}

In [4]:
var a = T.tensor2d([ [ 1, 7,10, 5],
                     [ 7, 2, 3,50],
                     [10, 3, 3, 5], 
                     [ 5,50, 5, 5] ])
console.log("matrix a")
a.print()
console.log("determinant")
var da = det(a)
console.log({da})

matrix a
Tensor
    [[1 , 7 , 10, 5 ],
     [7 , 2 , 3 , 50],
     [10, 3 , 3 , 5 ],
     [5 , 50, 5 , 5 ]]
determinant
{ da: 206050 }


In [5]:
a

Tensor {
  kept: false,
  isDisposedInternal: false,
  shape: [ 4, 4 ],
  dtype: 'float32',
  size: 16,
  strides: [ 4 ],
  dataId: {},
  id: 1,
  rankType: '2' }

In [6]:
// the inverse of the matrix : matrix adjoint method
function invertMatrix(m) {
    return T.tidy(() => {
        const d = det(m)
        if (d === 0) {
            return
        }
        const [D, _] = m.shape
        const rows = [...Array(D).keys()]
        let dets = []
        if(D === 2){
            dets.push
        }
        for (let i = 0; i < D; i++) {
            for (let j = 0; j < D; j++) {
                const sub_m = m.gather(T.tensor1d(rows.filter(e => e !== i), 'int32'))
                let sli
                if (j === 0) {
                    sli = sub_m.slice([0, 1], [D - 1, D - 1])
                } else if (j === D - 1) {
                    sli = sub_m.slice([0, 0], [D - 1, D - 1])
                } else {
                    const [a, b, c] = T.split(sub_m, [j, 1, D - (D + 1)], 1)
                    sli = T.concat([a, c], 1)
                }
                dets.push(Math.pow(-1, (i + j)) * det(sli))
            }
        }
        com = T.tensor2d(dets, [D, D])
        tr_com = com.transpose()
        inv_m = tr_com.div(T.scalar(d))
        
        return inv_m
    })
}

In [7]:
var a = T.tensor([[3.4638197, 0        , 0        ],
                  [0        , 3.1625874, 0        ],
                  [0        , 0        , 3.1625874]])
var i = invertMatrix(a)
console.log("inverse i")
i.print()
var prod = i.dot(a)
console.log("a * i")
prod.print()

Error: The sum of sizes must match the size of the axis dimension.

In [8]:
var w = T.tensor([[3.4638197, 0        ],
                  [0        , 3.1625874]])
invertMatrix(w).print()

Tensor
    [[0, 0],
     [0, 0]]


In [9]:
var m = T.tensor([[3,1,1],
                  [-1,3,1]])
var temp = m.dot(m.transpose())
var [S, U] = Eignh(temp)
S_ = S.pow(0.5).mul(T.eye(2))
S_.print()
iS = invertMatrix(S_)
iS.print()
V = m.transpose().dot(U).dot(iS)
S.print()
U.print()
V.print()

Tensor
    [[3.4638197, 0        ],
     [0        , 3.1625874]]
Tensor
    [[0, 0],
     [0, 0]]
Tensor
    [11.9980478, 10.0019588]
Tensor
    [[-0.7288829, 0.6846386 ],
     [-0.6846387, -0.7288826]]
Tensor
    [[0, 0],
     [0, 0],
     [0, 0]]
